<a href="https://colab.research.google.com/gist/ernestocl/c5d751734b469a8738785388fe093d3b/1-sales_calendar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este notebook sacamos las ventas por semana y unique id ==> "sales_calendar_v2"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import datetime
sns.set_theme(style='darkgrid')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Requerimiento:** analizar la situacion actual de las ventas de la compañía por:

* Producto -> Productos más vendidos y productos menos populares
* Tienda -> Precios por tienda (y añadir cantidades)
* Ciudad -> Mirar solo las de NY, Boston, Philly.

In [ ]:
sales = pd.read_csv("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD input/item_sales.csv")
calendar = pd.read_csv("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD input/daily_calendar_with_events.csv")

In [ ]:
#Creamos la lista de variables idenfiticadoras y variables de medidas.
lista = sales.columns.values.tolist()
lista_1 = lista[0:7]
del lista[0:7]

In [ ]:
#Utilizamos la función MELT
sales = sales.melt(id_vars=lista_1, var_name="date", value_name= "units")

In [ ]:
sales.head()

,id,item,category,department,store,store_code,region,date,units
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0


In [ ]:
sales.shape

(58327370, 9)

In [ ]:
calendar["date"] = pd.to_datetime(calendar["date"], format = "%Y-%m-%d")
calendar["week"] = calendar["date"].dt.isocalendar().week.astype(str)
calendar["year"] = calendar["date"].dt.isocalendar().year.astype(str)
calendar["year_week"] = calendar["year"] + "-" + calendar["week"]

In [ ]:
sales.rename(columns={"date": "d"}, inplace = True)

In [ ]:
sales.shape

(58327370, 9)

In [ ]:
sales_calendar = pd.merge(
    left = sales,
    right = calendar,
    how = 'left',
    on = ["d"])

In [ ]:
sales_calendar.head()

,id,item,category,department,store,store_code,region,d,units,date,weekday,weekday_int,event,week,year,year_week
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,4,2011,2011-4
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,4,2011,2011-4
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,4,2011,2011-4
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,4,2011,2011-4
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,4,2011,2011-4


In [ ]:
lista = sales_calendar.columns.values.tolist()
lista.remove("id")
lista.remove("units")
lista.remove("year_week")
lista

['item',
 'category',
 'department',
 'store',
 'store_code',
 'region',
 'd',
 'date',
 'weekday',
 'weekday_int',
 'event',
 'week',
 'year']

In [ ]:
sales_calendar.drop(lista, axis=1, inplace=True)

In [ ]:
sales_calendar.head()

,id,units,year_week
0,ACCESORIES_1_001_NYC_1,0,2011-4
1,ACCESORIES_1_002_NYC_1,0,2011-4
2,ACCESORIES_1_003_NYC_1,0,2011-4
3,ACCESORIES_1_004_NYC_1,0,2011-4
4,ACCESORIES_1_005_NYC_1,0,2011-4


In [ ]:
sales_calendar.shape

(58327370, 3)

In [ ]:
sales_calendar = sales_calendar.groupby(["year_week", "id"]).sum()

In [ ]:
sales_calendar.to_csv("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD output/sales_calendar_1.csv")

In [ ]:
sales_calendar

units
year_week id                            
2011-10   ACCESORIES_1_001_BOS_1       0
          ACCESORIES_1_001_BOS_2       0
          ACCESORIES_1_001_BOS_3       0
          ACCESORIES_1_001_NYC_1       0
          ACCESORIES_1_001_NYC_2       0
...                                  ...
2016-9    SUPERMARKET_3_827_NYC_3    167
          SUPERMARKET_3_827_NYC_4      0
          SUPERMARKET_3_827_PHI_1     36
          SUPERMARKET_3_827_PHI_2     21
          SUPERMARKET_3_827_PHI_3     15

[8354260 rows x 1 columns]